# Session 9: Coding Solution

## Emotion classification problem for Shopee comments

**Install `fasttext` for Pretrained Word Embedding**

In [ ]:
!pip install "git+https://github.com/facebookresearch/fastText.git"

  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-1i0lsjtu
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-1i0lsjtu
  Resolved https://github.com/facebookresearch/fastText.git to commit 440f46ac8811db0ce7ecb7dfb04f694453187db3
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4291600 sha256=5ce85a6bdf3dc2bae7e5b922e9d6e0e520bb2b04e31bf69edc6bb98f86b7f9dd
  Stored in directory: /tmp/pip-ephem-wheel-cache-sgv5go5h/wheels/3d/b7/8f/678f8b04408b579e2acad5f2416797aee4ebfe3529049a03b2
Successfully built fasttext


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML-intensive/data/sentiment_data.csv")
df.head()

,id,text,label
0,train_000000,Dung dc sp tot cam on shop Đóng_gói sản_phẩm r...,0
1,train_000001,Chất_lượng sản_phẩm tuyệt_vời . _Son mịn nhưng...,0
2,train_000002,Chất_lượng sản_phẩm tuyệt_vời nhưng k có hộp k...,0
3,train_000003,: ( ( Mình hơi thất_vọng 1 chút vì mình đã kỳ_...,1
4,train_000004,Lần trước mình mua áo_gió màu hồng rất ok mà đ...,1


Print a few compliments and a few critical comments

In [ ]:
print("Critical- LABEL = 1")
for text in df[df["label"] == 1]["text"].values[:10]:
  print(text)
print()
print("="*30)
print()
print("Compliment - LABEL = 0")
for text in df[df["label"] == 0]["text"].values[:10]:
  print(text)

Critical- LABEL = 1
: ( ( Mình hơi thất_vọng 1 chút vì mình đã kỳ_vọng cuốn sách khá nhiều hi_vọng nó sẽ nói về việc học_tập của cách sinh_viên trường Harvard ra sao những nỗ_lực của họ như_thế_nào 4h sáng ? tại_sao họ lại phải thức dậy vào thời_khắc đấy ? sau đó là cả một câu_chuyện ra sao . Cái mình thực_sự cần ở đây là câu_chuyện ẩn dấu trong đó để tự bản_thân mỗi người cảm_nhận và đi_sâu vào lòng người hơn . Còn cuốn sách này chỉ đơn_thuần là cuốn sách dạy kĩ_năng mà hầu_như sách nào cũng đã có . BUồn ...
Lần trước mình mua áo_gió màu hồng rất ok mà đợt này lại giao 2 cái áo_gió chất khác như vải mưa ý : ( (
Các siêu phẩm thấy cấu_hình toàn tựa tựa nhau . Ko còn sự đột_phá lớn nữa . Chỉ là nâng_cấp . Khác nhau về kiểu_dáng . Htc này vs giá trên thật lòng khó cạnh_tranh mạnh vs siêu phẩm khác , chỉ dành cho fan của họ thôi .
Đồng_hồ đẹp nhưng 1 cái đứt dây 1 cái k chạy mua ve phải sửa
Giao thiếu mình cái này rồi shop ơi T ^ T
không hài_lòng sản_phẩm cho lắm . giặt lan đầu_tiên da nh

We see that the above dataset has been applied Word Segment technique to separate words

However, the above dataset has not been cleaned (delete emoji, special characters, ...)

We will delete the special characters `: , = ...` but it should be noted not to delete the character `_` (which will spoil the result of the Word Segment)

In [ ]:
import re

def simple_preprocessing(text):
    # Remove emojis
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # emoticons
                               "\U0001F300-\U0001F5FF"  # symbols & pictographs
                               "\U0001F680-\U0001F6FF"  # transport & map symbols
                               "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "\U00002702-\U000027B0"  # Dingbats
                               "\U000024C2-\U0001F251"  # Enclosed characters
                               "]+", flags=re.UNICODE)

    text = emoji_pattern.sub(r'', text)
    # Remove special characters excluding underscore (_) with regex python
    text = re.sub(r'[^\w\s_]', '', text)

    '''
    Deleting special characters can lead to the excess spaces
    for example "huhu : ( (  buồn quá" sẽ thành "huhu     buồn quá"
    We will split the text in a space and then join it again to correct this case
    '''

    text = " ".join(text.split())
    text = text.strip().lower()
    return text

df["text"] = df["text"].apply(simple_preprocessing)

In [ ]:
# Review results after preprocessing
print("Critical- LABEL = 1")
for text in df[df["label"] == 1]["text"].values[:10]:
  print(text)
print()
print("="*30)
print()
print("Compliment - LABEL = 0")
for text in df[df["label"] == 0]["text"].values[:10]:
  print(text)

Critical- LABEL = 1
mình hơi thất_vọng 1 chút vì mình đã kỳ_vọng cuốn sách khá nhiều hi_vọng nó sẽ nói về việc học_tập của cách sinh_viên trường harvard ra sao những nỗ_lực của họ như_thế_nào 4h sáng tại_sao họ lại phải thức dậy vào thời_khắc đấy sau đó là cả một câu_chuyện ra sao cái mình thực_sự cần ở đây là câu_chuyện ẩn dấu trong đó để tự bản_thân mỗi người cảm_nhận và đi_sâu vào lòng người hơn còn cuốn sách này chỉ đơn_thuần là cuốn sách dạy kĩ_năng mà hầu_như sách nào cũng đã có buồn
lần trước mình mua áo_gió màu hồng rất ok mà đợt này lại giao 2 cái áo_gió chất khác như vải mưa ý
các siêu phẩm thấy cấu_hình toàn tựa tựa nhau ko còn sự đột_phá lớn nữa chỉ là nâng_cấp khác nhau về kiểu_dáng htc này vs giá trên thật lòng khó cạnh_tranh mạnh vs siêu phẩm khác chỉ dành cho fan của họ thôi
đồng_hồ đẹp nhưng 1 cái đứt dây 1 cái k chạy mua ve phải sửa
giao thiếu mình cái này rồi shop ơi t t
không hài_lòng sản_phẩm cho lắm giặt lan đầu_tiên da nhoe màu hết rồi
đồng_hồ thì đẹp thật nhưng 

### Train Test Split

We will split the dataset into 3 sets
- Train
- Validation
- Test

You should note: when performing Tokenizer and Embedding, you can only manipulate on the Train episode. This leads to when testing the model on the Validation and Test set will be able to occur the following cases:
- In 2 episodes of Val-Test appear words that never appeared in the episode Train
- In 2 episodes of Val-Test appear documents that are too long or too short for Train

The above differences will lower the performance of model → this is the reason that you need to have 1 quality dataset so that the model can run well in real application


In [ ]:
from sklearn.model_selection import train_test_split

sentences, labels = df["text"].values, df["label"].values
x_train, x_val, y_train, y_val = train_test_split(
    sentences,
    labels,
    test_size=0.4,
    shuffle=True,
    random_state=42,
    stratify=labels
)

x_val, x_test, y_val, y_test = train_test_split(
    x_val,
    y_val,
    test_size=0.5,
    shuffle=True,
    random_state=42,
    stratify=y_val
)

print("Train Set")
print(x_train.shape, y_train.shape)
print("Validation Set")
print(x_val.shape, y_val.shape)
print("Test Set")
print(x_test.shape, y_test.shape)

Train Set
(9651,) (9651,)
Validation Set
(3217,) (3217,)
Test Set
(3218,) (3218,)


### Tokenizer

In this Assignment, we will use the `TextVectorization` layer of `tensorflow.keras` to turn the Tokenizer operation into 1 part of the model (in the Lab article, we Tokenizer first and then put it into the model)

Before initializing the Tokenizer, we need to calculate the length and number of unique words of the documents in the Train set.

**Count unique words**

In [ ]:
word_set = set()
for text in x_train:
  words = text.split()
  for word in words:
    if word not in word_set:
      word_set.add(word)

VOCAB_SIZE = len(word_set)
print(VOCAB_SIZE)

9930


**Count the average length of eachg text**

In [ ]:
count_word = []
for text in x_train:
  words = text.split()
  count_word.append(len(words))

min(count_word), max(count_word), sum(count_word)/len(count_word)

(0, 369, 17.793907367112215)

We see a problem when the value `min` is zero.

In [ ]:
for text in x_train:
  if text == "":
    print(True)

True
True
True


Delete lines with null values in `x_train` and delete corresponding lines in `y_train`

In [ ]:
new_train_text = []
new_train_label = []

for text, label in zip(x_train, y_train):
  if text != "":
    new_train_text.append(text)
    new_train_label.append(label)

x_train = np.array(new_train_text)
y_train = np.array(new_train_label)

In [ ]:
count_word = []
for text in x_train:
  words = text.split()
  count_word.append(len(words))

min(count_word), max(count_word), sum(count_word)/ len(count_word)

(1, 369, 17.799440298507463)

Do the same with test and validation sets

In [ ]:
new_test_text = []
new_test_label = []

for text, label in zip(x_test, y_test):
  if text != "":
    new_test_text.append(text)
    new_test_label.append(label)

x_test = np.array(new_test_text)
y_test = np.array(new_test_label)

In [ ]:
new_val_text = []
new_val_label = []

for text, label in zip(x_val, y_val):
  if text != "":
    new_val_text.append(text)
    new_val_label.append(label)

x_val = np.array(new_val_text)
y_val = np.array(new_val_label)

In fact, the `TextVectorization` layer will automatically calculate the number of unique words (plus 2 for `out_of_vocab` and `special` tokens)

In [ ]:
from tensorflow.keras.layers import TextVectorization

# We calculate 9930, here we leave 10k to see the result of the layer
VOCAB_SIZE = 10000
MAX_LENGTH = 50 # average length is 17

tokenizer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    standardize=None, # preprocessing already done above
    output_mode="int", # output list containing tokreprocess (representing words in the field)
    output_sequence_length=MAX_LENGTH # padding or truncate until MAX_LENGTH
)

# Fit on x_train
tokenizer.adapt(x_train)
print(tokenizer.vocabulary_size())

9932


Update the variable `VOCAB_SIZE`

In [ ]:
VOCAB_SIZE = tokenizer.vocabulary_size() # 9932

### Pretrained Word Embedding

In [ ]:
import fasttext

ft = fasttext.load_model("/content/drive/MyDrive/Colab Notebooks/ML-intensive/data/cc.vi.50.bin")

This pretrained Word Embedding will represent the word with 1 vector `50 dimensions`

In [ ]:
text = "đẹp_trai"
embedding = ft[text]
print(embedding)
print(embedding.shape)

[-0.08987515  0.11855838 -0.041201   -0.08122422 -0.04498195 -0.07693475
  0.01601385 -0.10124682  0.03174876  0.03258704  0.02468052  0.00593183
 -0.01430611  0.04422166  0.02866276 -0.01971636  0.01052842  0.01453072
 -0.01432774 -0.08909895 -0.06839065 -0.03624108  0.06818447  0.06247804
 -0.11234589  0.06176881  0.04748557 -0.02569062 -0.02754932  0.00407921
 -0.00562474 -0.04759934 -0.01732392  0.03452192  0.01076003  0.02103091
  0.03082983  0.03875627  0.00951283 -0.03213377 -0.04992865  0.00212197
 -0.11983471 -0.00709106 -0.01203236 -0.01266732 -0.04353423 -0.00745085
  0.03781988 -0.0106875 ]
(50,)


#### TODO 1

We will write the algorithm described in the Pre-Class article to derive vector embedding for each word in the dictionary
- Initalize empty  list `embeddings`
- Loop through each word in the dictionary
  - Retrieve a list of words to repeat with `tokenizer.get_vocabulary(include_special_tokens=True)`
  - Use `tqdm` to display progress bar `tqdm(tokenizer.get_vocabulary(include_special_tokens=True))`
- if the word exists in Pretrained Embedding (use the `in ft` command to check)
  - add it to `embeddings`
- If not exist
  - Intialize randomly a vector of 50 features `np.random.uniform`, ranging from `-0.05` to `0.05` and then add into `embeddings`
- Convert `embeddings` into `numpy array` and print the shape to test.

In [ ]:
len(tokenizer.get_vocabulary(include_special_tokens=False))

9930

In [ ]:
len(tokenizer.get_vocabulary(include_special_tokens=True))

9932

In [ ]:
if "đẹp_trai" in ft:
  print(True)
else:
  print(False)

True


In [ ]:
from tqdm.notebook import tqdm
embeddings = []

for word in tqdm(tokenizer.get_vocabulary(include_special_tokens=True)):
  if word in ft:
    embeddings.append(ft[word])
  else:
    embeddings.append(np.random.uniform(low=-0.5, high=0.5, size=(50,)))

embeddings = np.array(embeddings)
embeddings.shape

  0%|          | 0/9932 [00:00<?, ?it/s]

(9932, 50)

Initialize layer `Embedding`

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import Constant # to initialize the available Embedding = weight layer

# Include the embeddings variable as the weight of the layer
embedding_layer = Embedding(
    VOCAB_SIZE,
    50,
    embeddings_initializer=Constant(embeddings), # plug in the embeddings variable as the weight of the layer
    name="embedding" # Name the layer
)

### Simple Recurrent Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, SimpleRNN, LSTM, GRU
from tensorflow import string


'''
Pipeline:
- input layer receive documents (shape=1, dtype=str)
- tokenizer
- embedding
- RNN & MLP
'''

model = Sequential()

model.add(Input(shape=(1,), dtype=string))
xmodel.add(tokenizer)
model.add(embedding_layer)
model.add(SimpleRNN(32))

model.add(Dense(16, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 50)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 50, 50)            496600    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                2656      
                                                                 
 dense (Dense)               (None, 16)                528       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 499,801
Trainable params: 499,801
Non-trainable params: 0
__________________________________________________

In [ ]:
model.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['accuracy'
              ])

model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=5
)

Epoch 1/5
302/302 [==============================] - 64s 191ms/step - loss: 0.6701 - accuracy: 0.6015 - val_loss: 0.6665 - val_accuracy: 0.6052
Epoch 2/5
302/302 [==============================] - 35s 116ms/step - loss: 0.6596 - accuracy: 0.6106 - val_loss: 0.6616 - val_accuracy: 0.6080
Epoch 3/5
302/302 [==============================] - 28s 92ms/step - loss: 0.6454 - accuracy: 0.6188 - val_loss: 0.6674 - val_accuracy: 0.6049
Epoch 4/5
302/302 [==============================] - 29s 95ms/step - loss: 0.6361 - accuracy: 0.6251 - val_loss: 0.6792 - val_accuracy: 0.6040
Epoch 5/5
302/302 [==============================] - 29s 97ms/step - loss: 0.6266 - accuracy: 0.6358 - val_loss: 0.6035 - val_accuracy: 0.7128


In [ ]:
model.evaluate(x_test, y_test)
y_pred_test = model.predict(x_test) >= 0.5

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(confusion_matrix(y_test, y_pred_test), annot=True, fmt="d")
plt.show()
print(classification_report(y_test, y_pred_test))

#### TODO 2

Now it`s your turn, use more complex models:
1. Stacked Biderectional RNN: oftenly, only 2 to 3 layers should be stacked
2. Replace the RNN in model 1 with LSTM or GRU and train again

**Note: you need to reinitialize the 'Embedding' layer from the 'embeddings' variable when creating a new model (as the current 'Embedding' layer has already been trained)**

There are 1 techniques to train the model better when using the Pretrained Model::
- Freezing the pretrained layer (in this tutorial the `Embedding` layer), i.e. not updating the weight of this layer during training
- Train the model
- Open the pretrained layers, and continue training with a small `learning_rate`.

We will learn about the above technique later, but you can try it

```python
# Initialize embedding
embedding_layer = Embedding(
    VOCAB_SIZE,
    50,
    embeddings_initializer=Constant(embeddings),
    name="embedding"
)

# Freeze weights
embedding_layer.trainable = False

# Create the model
# Start training process
...
# Unfreeze weights
embedding_layer.trainable = True

# re-compile model (compile only, not recreate) and use smaller learning rate
model.compile(optimizer=Adam(learning_rate=smaller_lr))
model.fit(...)
```

In [ ]:
# YOUR SOLUTION
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow import string
from tensorflow.keras.optimizers import Adam

# Intialize Embedding layer
embedding_layer = layers.Embedding(
    VOCAB_SIZE,
    50,
    embeddings_initializer = Constant(embeddings),
    name="embedding"
)

# Freeze weights
embedding_layer.trainble = False

# Create the model
model = Sequential()
model.add(layers.Input(shape=(1,), dtype=string))
model.add(tokenizer)
model.add(embedding_layer)

# Due to stacking to layers LSTM -> return_sequences=True is required in the first layer
model.add(layers.Bidirectional(layers.LSTM(32, return_sequences=True)))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(Dense(16, activation="relu"))
model.add(Dense(1, activation='sigmoid'))

# compile model
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5)

# Unfreeze Embeddding
embedding_layer.trainable = True

# optimizer
adam = Adam(0.0005)

# re-compile and re-fit the model with smaller learning rate
model.compile(optimizer=adam, loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5)

Epoch 1/5
302/302 [==============================] - 43s 100ms/step - loss: 0.3946 - accuracy: 0.8164 - val_loss: 0.2911 - val_accuracy: 0.8834
Epoch 2/5
302/302 [==============================] - 12s 39ms/step - loss: 0.2331 - accuracy: 0.9101 - val_loss: 0.2551 - val_accuracy: 0.8949
Epoch 3/5
302/302 [==============================] - 9s 30ms/step - loss: 0.1778 - accuracy: 0.9334 - val_loss: 0.2733 - val_accuracy: 0.8934
Epoch 4/5
302/302 [==============================] - 8s 27ms/step - loss: 0.1409 - accuracy: 0.9516 - val_loss: 0.2961 - val_accuracy: 0.8893
Epoch 5/5
302/302 [==============================] - 7s 22ms/step - loss: 0.1174 - accuracy: 0.9621 - val_loss: 0.3195 - val_accuracy: 0.8813
Epoch 1/5
302/302 [==============================] - 43s 110ms/step - loss: 0.0920 - accuracy: 0.9700 - val_loss: 0.3745 - val_accuracy: 0.8841
Epoch 2/5
302/302 [==============================] - 12s 38ms/step - loss: 0.0769 - accuracy: 0.9755 - val_loss: 0.4313 - val_accuracy: 0.8847
